## Ablation Experiment

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
from copy import deepcopy
import os
import numpy as np
import pandas as pd
import torch

from lib.problems import ProblemDataset, Generator, eval_cop, sub_sample
from lib.utils import CCPInstance
from baselines.utils import eval_method
from baselines.CPP import methods_registry
from baselines.CPP.methods_registry import cap_kmeans
from lib.ltr.utils import load_model
from lib.ltr.ccp.method import CKMeans
from lib.ltr.ccp.method_no_alt_assign import NCCNA

### Shanghai

In [ ]:
DSET = "shanghai_telecom" #
CAP = 1.1
DATA_PTH = f"data/CCP/benchmark/{DSET}/data_norm_cap{CAP}_.dat"
COLS = ["num_users", "x_coord", "y_coord", "normalized_workload"]
CKPT = "outputs/final/ccp_200/gnn_pool_pointwise/2023-01-09_11-19-49_057495/checkpoints/epoch=198_val_acc=0.9858.ckpt"

SAMPLING_METHOD = "quadrant_rnd"
TRGT_SIZE = 50
SIZE = 250

SEED = 999
N = 200
WF = (1.5, 4.0)
FULL_K = 40
MAX_W = 0.2
INF = float("inf")

save_pth = f"./data/CCP/ablation/{DSET}_{N}"
fname = f"abl_n{N}_s{TRGT_SIZE}_cap{str(CAP).replace('.', '_')}_seed{SEED}"

os.makedirs(save_pth, exist_ok=True)

In [ ]:
# load original data
data = torch.load(DATA_PTH)
print(len(data))
xyw = data[:, 1:]
rng = np.random.RandomState(SEED)


In [ ]:
samples = sub_sample(
    deepcopy(xyw),
    size=SIZE,
    n=N,
    rng=rng,
    method=SAMPLING_METHOD,
    weight_factor=WF,
    max_weight=MAX_W
)
# convert to CCPInstances
instances = [
    CCPInstance(
        coords=s[:, :2],
        demands=s[:, -1],
        graph_size=N,
        constraint_value=1.0,
        )
    for s in samples
]



In [ ]:
K_NOT_KNOWN = False
CUDA = False
SEED = 42
NSEEDS = 1
NUM_INIT = 8
ITERS = 50
TOL = 0.0   # no early stopping
INIT_METHODS = [
    "topk", # originally proposed by Geetha et al. 2009
    "k-means++", # Arthur and Vassilvitskii, 2006
    "ckm++", # ours
]
SAVE_DIR = os.path.join(save_pth, fname)

In [ ]:
metrics = {}
RESULTS = {}
seeds = [SEED]
ds = ProblemDataset(problem="ccp", seed=SEED)
ds.size = len(instances)
ds.data = instances


In [ ]:
## reload

def to_instance(d: dict):
    d.pop("problem")
    d.pop("size")
    gs = int(d['graph_size'])
    d['graph_size'] = gs
    d['constraint_value'] = float(d['constraint_value'])
    d['num_components'] = int(d['num_components'])
    d['coords'] = d['coords'].reshape(gs, 2)
    d['demands'] = d['demands'].reshape(gs)
    return CCPInstance(**d)

all_res = torch.load("data/CCP/ablation/shanghai_telecom_200/abl_n200_s50_cap1_1_seed999/ncc_greedy/eval_results_full_1.pkl")
data = [to_instance(e['instance']) for e in all_res]
ds.size = len(data)
ds.data = data
ds.data[0]

In [ ]:
mthd = "random_select"
result, smry = eval_method(
    method=getattr(methods_registry, mthd),
    dataset=ds,
    seeds=seeds,
    save_dir=SAVE_DIR,
    cuda=False,
    k_not_known=True,
    method_str=mthd,
    verbose=False,
)
rnd_res = deepcopy(result)

# retrieve the respective k values and set them for the instances
# get the minimum k value found by the random method for all seeds
k_vals = np.array([r['nc'] for r in rnd_res]).reshape(NSEEDS, -1).min(axis=0)
data = ds.data.copy()
assert len(k_vals) == len(data), f"{len(k_vals)} != {len(data)}"
for i in range(len(data)):
    data[i] = data[i].update(num_components=int(k_vals[i]))
ds.data = data

In [ ]:
init_methods = INIT_METHODS + INIT_METHODS[1:]
num_inits = [1] * 3 + [NUM_INIT] * 2

mthd = "cap_kmeans"
infs = {}

for init_mthd, num_init in zip(init_methods, num_inits):
    m_id = f"{mthd}_{init_mthd}-{num_init}{'_cuda' if CUDA else ''}"
    print(f"running for {m_id}...")

    result, smry = eval_method(
        method=getattr(methods_registry, mthd),
        dataset=ds,
        seeds=seeds,
        save_dir=SAVE_DIR,
        cuda=CUDA,
        k_not_known=K_NOT_KNOWN,
        verbose=False,
        num_init=num_init,
        max_iter=ITERS,
        tol=TOL,
        init_method=init_mthd,
    )

    RESULTS[m_id] = result
    # check if there are any infeasible instances and replace cost with base cost
    res = deepcopy(result)
    costs = np.array([r['tot_center_dist'] for r in res])
    costs_ = costs.reshape(NSEEDS, -1)
    inf_idx = []
    for i in range(len(ds)):
        inst_cost = costs_[:, i]
        inf_msk = inst_cost == INF
        if np.any(inf_msk):
            inf_idx.append(i)

    infs[m_id] = {"cost": costs, "summary": smry, "inf_idx": inf_idx}

In [ ]:
# find TRGT_SIZE instances which were feasibly solved by all init methods of cap kmeans
unq_inf_idx = set()
for v in infs.values():
    for e in v["inf_idx"]:
        unq_inf_idx.add(e)
print(len(unq_inf_idx))
valid_idx = [i for i in range(SIZE) if i not in unq_inf_idx]
print(valid_idx)
assert len(valid_idx) >= TRGT_SIZE
valid_idx = valid_idx[:TRGT_SIZE]
print(len(valid_idx), valid_idx)

In [ ]:
for k, v in infs.items():
    # read out results only for valid idx
    smry = v['summary']
    costs = v['cost']
    #print(costs)
    costs = np.array([c for i, c in enumerate(costs) if i in valid_idx])
    #print(costs)
    costs = costs.reshape(NSEEDS, -1)
    smry['center_dist_mean'] = np.mean(costs)
    smry['center_dist_std'] = np.mean(np.std(costs, axis=0))
    rt = [e['run_time'] for e in RESULTS[k]]
    rt = [c for i, c in enumerate(rt) if i in valid_idx]
    smry['run_time_mean'] = np.mean(rt)
    m_id = f"{mthd}_{k}{'_cuda' if CUDA else ''}"
    print(f"{m_id} summary: {smry}")
    #metrics[m_id] = smry



In [ ]:
# reduce dataset for all other methods to valid_idx selection
ds.size = len(valid_idx)
ds.data = [c for i, c in enumerate(ds.data) if i in valid_idx]


In [ ]:
# this setup corresponds to the cap_kmeans method
# with an alternative order of assignment,
# i.e. instead of sequentially assigning the node with the
# highest priority to the closest center,
# we cycle through the centers and sequentially add the highest
# priority node for that specific center to its cluster
# (vanilla_priority=True -> it does not use the NN model at all!)

mthd = "cap_kmeans_alt"
model = load_model("ccp", CKPT)

for init_mthd in INIT_METHODS:
    m_id = f"{mthd}_{init_mthd}{'_cuda' if CUDA else ''}"

    ckmeans = CKMeans(
        model=model,
        seed=SEED,
        verbose=False,
        ###
        num_init=1 if init_mthd == "topk" else NUM_INIT,
        max_iter=ITERS,
        tol=TOL,
        init_method=init_mthd,
        ###
        vanilla_priority=True,
        opt_last_frac=0.0,
        opt_last_samples=0,
        opt_last_prio=False,
    )

    result, smry = eval_method(
        method=ckmeans.inference,
        dataset=ds,
        seeds=seeds,
        save_dir=SAVE_DIR,
        cuda=CUDA,
        k_not_known=K_NOT_KNOWN,
        method_str=mthd,
    )
    RESULTS[m_id] = result
    # check if there are any infeasible instances and replace cost with base cost
    res = deepcopy(result)
    costs = np.array([r['tot_center_dist'] for r in res])
    costs = costs.reshape(NSEEDS, -1)
    smry['center_dist_mean'] = np.mean(costs)
    smry['center_dist_std'] = np.mean(np.std(costs, axis=0))
    print(f"{m_id} summary: {smry}")
    metrics[m_id] = smry

In [ ]:
# corresponds to Cap-Kmeans using the NN model (neural scoring function)
# for priority estimation but without alternative assignment

mthd = "cap_kmeans_nsf"
model = load_model("ccp", CKPT)

for init_mthd in INIT_METHODS:
    m_id = f"{mthd}_{init_mthd}{'_cuda' if CUDA else ''}"

    ckmeans = NCCNA(
        model=model,
        seed=SEED,
        verbose=False,
        ###
        num_init=1 if init_mthd == "topk" else NUM_INIT,
        max_iter=ITERS,
        tol=TOL,
        init_method=init_mthd,
        ###
        vanilla_priority=False,
        opt_last_frac=0.0,
        opt_last_samples=0,
        opt_last_prio=False,
    )

    result, smry = eval_method(
        method=ckmeans.inference,
        dataset=ds,
        seeds=seeds,
        save_dir=SAVE_DIR,
        cuda=CUDA,
        k_not_known=K_NOT_KNOWN,
        method_str=mthd,
    )
    RESULTS[m_id] = result
    # check if there are any infeasible instances and replace cost with base cost
    res = deepcopy(result)
    costs = np.array([r['tot_center_dist'] for r in res])
    costs = costs.reshape(NSEEDS, -1)
    smry['center_dist_mean'] = np.mean(costs)
    smry['center_dist_std'] = np.mean(np.std(costs, axis=0))
    print(f"{m_id} summary: {smry}")
    metrics[m_id] = smry

In [ ]:
# greedily assigns the last 'opt_last_frac' fraction of total nodes
# ordered by their absolute priority to the closest center

mthd = "ncc_greedy"
model = load_model("ccp", CKPT)

for init_mthd in INIT_METHODS:
    m_id = f"{mthd}_{init_mthd}{'_cuda' if CUDA else ''}"

    ckmeans = CKMeans(
        model=model,
        seed=SEED,
        verbose=False,
        ###
        num_init=1 if init_mthd == "topk" else NUM_INIT,
        max_iter=ITERS,
        tol=TOL,
        init_method=init_mthd,
        ###
        vanilla_priority=False,
        opt_last_frac=0.7,
        opt_last_samples=1, # no multiple samples
        opt_last_prio=True
    )

    result, smry = eval_method(
        method=ckmeans.inference,
        dataset=ds,
        seeds=seeds,
        save_dir=SAVE_DIR,
        cuda=CUDA,
        k_not_known=K_NOT_KNOWN,
        method_str=mthd,
    )
    RESULTS[m_id] = result
    # check if there are any infeasible instances and replace cost with base cost
    res = deepcopy(result)
    costs = np.array([r['tot_center_dist'] for r in res])
    costs = costs.reshape(NSEEDS, -1)
    smry['center_dist_mean'] = np.mean(costs)
    smry['center_dist_std'] = np.mean(np.std(costs, axis=0))
    print(f"{m_id} summary: {smry}")
    metrics[m_id] = smry

In [ ]:
# samples multiple assignments for the last 'opt_last_frac' fraction of total nodes
# and selects the best one

mthd = "ncc_samp"
model = load_model("ccp", CKPT)
#init_mthd = "c2km++"
for init_mthd in INIT_METHODS:
    m_id = f"{mthd}_{init_mthd}{'_cuda' if CUDA else ''}"

    ckmeans = CKMeans(
        model=model,
        seed=SEED,
        verbose=False,
        ###
        num_init=1 if init_mthd == "topk" else NUM_INIT,
        max_iter=ITERS,
        tol=TOL,
        init_method=init_mthd,
        ###
        vanilla_priority=False,
        opt_last_frac=0.7,
        opt_last_samples=64,
        opt_last_prio=True
    )

    result, smry = eval_method(
        method=ckmeans.inference,
        dataset=ds,
        seeds=seeds,
        save_dir=SAVE_DIR,
        cuda=CUDA,
        k_not_known=K_NOT_KNOWN,
        method_str=mthd,
    )
    RESULTS[m_id] = result
    # check if there are any infeasible instances and replace cost with base cost
    res = deepcopy(result)
    costs = np.array([r['tot_center_dist'] for r in res])
    costs = costs.reshape(NSEEDS, -1)
    smry['center_dist_mean'] = np.mean(costs)
    smry['center_dist_std'] = np.mean(np.std(costs, axis=0))
    print(f"{m_id} summary: {smry}")
    metrics[m_id] = smry

In [ ]:
# convert to dataframe for nice table ;)
metric_df = pd.DataFrame(metrics)
metric_df


In [ ]:
# !!!
# opt_last_frac=0.99, opt_last_samples=16
# cap_kmeans_nsf_topk summary: {'center_dist_mean': 0.5042707053562993, 'center_dist_std': 0.0, 'nc_mean': 8.66, 'nc_std': 2.5108564275959706, 'nc_median': 9.0, 'run_time_mean': 1.0674996393000766, 'run_time_total': 53.374981965003826, 'num_infeasible': 0}
# cap_kmeans_nsf_k-means++ summary: {'center_dist_mean': 0.46759631351147113, 'center_dist_std': 0.0, 'nc_mean': 8.66, 'nc_std': 2.5108564275959706, 'nc_median': 9.0, 'run_time_mean': 2.732793123839656, 'run_time_total': 136.6396561919828, 'num_infeasible': 0}
# cap_kmeans_nsf_ckm++ summary: {'center_dist_mean': 0.46429420882987116, 'center_dist_std': 0.0, 'nc_mean': 8.66, 'nc_std': 2.5108564275959706, 'nc_median': 9.0, 'run_time_mean': 2.7289918054996813, 'run_time_total': 136.44959027498408, 'num_infeasible': 0}

# opt_last_frac=0.90, opt_last_samples=16
# cap_kmeans_nsf_ckm++ summary: {'center_dist_mean': 0.44793172402515713, 'center_dist_std': 0.0, 'nc_mean': 8.66, 'nc_std': 2.5108564275959706, 'nc_median': 9.0, 'run_time_mean': 2.68136288738031, 'run_time_total': 134.0681443690155, 'num_infeasible': 0}
# opt_last_frac=0.80, opt_last_samples=16
# cap_kmeans_nsf_ckm++ summary: {'center_dist_mean': 0.44748048922138345, 'center_dist_std': 0.0, 'nc_mean': 8.66, 'nc_std': 2.5108564275959706, 'nc_median': 9.0, 'run_time_mean': 2.618354005799847, 'run_time_total': 130.91770028999235, 'num_infeasible': 0}
# opt_last_frac=0.75, opt_last_samples=16
# cap_kmeans_nsf_ckm++ summary: {'center_dist_mean': 0.4469523817082363, 'center_dist_std': 0.0, 'nc_mean': 8.66, 'nc_std': 2.5108564275959706, 'nc_median': 9.0, 'run_time_mean': 2.6313813093400675, 'run_time_total': 131.56906546700338, 'num_infeasible': 0}
# opt_last_frac=0.70, opt_last_samples=16
# cap_kmeans_nsf_ckm++ summary: {'center_dist_mean': 0.44451487442761617, 'center_dist_std': 0.0, 'nc_mean': 8.66, 'nc_std': 2.5108564275959706, 'nc_median': 9.0, 'run_time_mean': 2.6363163709201762, 'run_time_total': 131.8158185460088, 'num_infeasible': 0}
# opt_last_frac=0.60, opt_last_samples=16
# cap_kmeans_nsf_ckm++ summary: {'center_dist_mean': 0.45103828188218453, 'center_dist_std': 0.0, 'nc_mean': 8.66, 'nc_std': 2.5108564275959706, 'nc_median': 9.0, 'run_time_mean': 2.81216765499998, 'run_time_total': 140.608382749999, 'num_infeasible': 0}

# opt_last_frac=0.70, opt_last_samples=16
# cap_kmeans_nsf_ckm++ summary: {'center_dist_mean': 0.44451487442761617, 'center_dist_std': 0.0, 'nc_mean': 8.66, 'nc_std': 2.5108564275959706, 'nc_median': 9.0, 'run_time_mean': 2.6363163709201762, 'run_time_total': 131.8158185460088, 'num_infeasible': 0}
# opt_last_frac=0.70, opt_last_samples=32
# cap_kmeans_nsf_ckm++ summary: {'center_dist_mean': 0.4442066230780878, 'center_dist_std': 0.0, 'nc_mean': 8.66, 'nc_std': 2.5108564275959706, 'nc_median': 9.0, 'run_time_mean': 3.4557080210199955, 'run_time_total': 172.78540105099978, 'num_infeasible': 0}


mthd = "cap_kmeans_nsf"
model = load_model("ccp", CKPT)

for init_mthd in INIT_METHODS[-1:]:
    m_id = f"{mthd}_{init_mthd}{'_cuda' if CUDA else ''}"

    ckmeans = CKMeans(
        model=model,
        seed=SEED,
        verbose=False,
        ###
        num_init=1 if init_mthd == "topk" else NUM_INIT,
        max_iter=ITERS,
        tol=TOL,
        init_method=init_mthd,
        ###
        vanilla_priority=False,
        opt_last_frac=0.7,
        opt_last_samples=32,
        opt_last_prio=True,
    )

    result, smry = eval_method(
        method=ckmeans.inference,
        dataset=ds,
        seeds=seeds,
        save_dir=SAVE_DIR,
        cuda=CUDA,
        k_not_known=K_NOT_KNOWN,
        method_str=mthd,
    )
    RESULTS[m_id] = result
    # check if there are any infeasible instances and replace cost with base cost
    res = deepcopy(result)
    costs = np.array([r['tot_center_dist'] for r in res])
    costs = costs.reshape(NSEEDS, -1)
    smry['center_dist_mean'] = np.mean(costs)
    smry['center_dist_std'] = np.mean(np.std(costs, axis=0))
    print(f"{m_id} summary: {smry}")
    metrics[m_id] = smry

In [ ]:
mthd = "ncc_v2_greedy"
model = load_model("ccp", CKPT)

for init_mthd in INIT_METHODS[-1:]:
    m_id = f"{mthd}_{init_mthd}{'_cuda' if CUDA else ''}"

    ckmeans = NCC(
        model=model,
        seed=SEED,
        verbose=False,
        ###
        num_init=1 if init_mthd == "topk" else NUM_INIT,
        max_iter=ITERS,
        tol=TOL,
        init_method=init_mthd,
        ###
        vanilla_priority=False,
        num_samples=1,
    )

    result, smry = eval_method(
        method=ckmeans.inference,
        dataset=ds,
        seeds=seeds,
        save_dir=SAVE_DIR,
        cuda=CUDA,
        k_not_known=K_NOT_KNOWN,
        method_str=mthd,
    )
    RESULTS[m_id] = result
    # check if there are any infeasible instances and replace cost with base cost
    res = deepcopy(result)
    costs = np.array([r['tot_center_dist'] for r in res])
    costs = costs.reshape(NSEEDS, -1)
    smry['center_dist_mean'] = np.mean(costs)
    smry['center_dist_std'] = np.mean(np.std(costs, axis=0))
    print(f"{m_id} summary: {smry}")
    metrics[m_id] = smry


In [ ]:
mthd = "ncc_v2_samp"
model = load_model("ccp", CKPT)

for init_mthd in INIT_METHODS[-1:]:
    m_id = f"{mthd}_{init_mthd}{'_cuda' if CUDA else ''}"

    ckmeans = NCC(
        model=model,
        seed=SEED,
        verbose=False,
        ###
        num_init=1 if init_mthd == "topk" else NUM_INIT,
        max_iter=ITERS,
        tol=TOL,
        init_method=init_mthd,
        ###
        vanilla_priority=False,
        num_samples=64,
    )

    result, smry = eval_method(
        method=ckmeans.inference,
        dataset=ds,
        seeds=seeds,
        save_dir=SAVE_DIR,
        cuda=CUDA,
        k_not_known=K_NOT_KNOWN,
        method_str=mthd,
    )
    RESULTS[m_id] = result
    # check if there are any infeasible instances and replace cost with base cost
    res = deepcopy(result)
    costs = np.array([r['tot_center_dist'] for r in res])
    costs = costs.reshape(NSEEDS, -1)
    smry['center_dist_mean'] = np.mean(costs)
    smry['center_dist_std'] = np.mean(np.std(costs, axis=0))
    print(f"{m_id} summary: {smry}")
    metrics[m_id] = smry